<a href="https://colab.research.google.com/github/harshyadavv2456/Mnemos-2.0/blob/main/colab_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNEMOS 2.1 – Institutional-Grade Market Intelligence (Colab)

**Runs 24/7 on free Colab.** NSE stocks + Nifty, Sensex, Bank Nifty, Nifty IT + Gold/Silver. Friction + GROQ Llama analysis, daily heartbeat, reports. Mail trail: comma-separated ALERT_EMAIL_TO.

1. **Run Section 1** (deps), then **Section 1.5** (clone repo from GitHub into Colab). When opening from "Open in Colab", the project is not on disk—Section 1.5 clones it so `engine` and other modules are found.
2. **Secrets**: Add in Colab Secrets (key icon) or paste in Section 2: TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID, GMAIL_USER, GMAIL_APP_PASSWORD, ALERT_EMAIL_TO, GROQ_API_KEY. Enable "Notebook access" for each secret.
3. **Run all** – Sections 1 → 1.5 → 2 → 3 → 4. Last cell runs MNEMOS 24/7. Re-run the last cell if the session drops.

**Keepalive**: Keep tab active; optional: add a cell with `%%javascript` and run every ~60 min:
`document.querySelector('.colab-connect-button')?.click();`

### How to add secrets (all secure – never in the repo)

**Option A – Colab Secrets (recommended)**  
1. Click the **key icon** in the left sidebar (Secrets).  
2. Add each secret; for each one turn **ON** "Notebook access" so this notebook can read it.
3. Names and where to get the value:

- **TELEGRAM_BOT_TOKEN** – from @BotFather on Telegram  
- **TELEGRAM_CHAT_ID** – from @userinfobot (your numeric ID)  
- **GMAIL_USER** – your Gmail address  
- **GMAIL_APP_PASSWORD** – Google Account → Security → 2-Step Verification → App passwords (16 characters, not your login password)  
- **ALERT_EMAIL_TO** – same as GMAIL_USER, or comma-separated: `a@gmail.com,b@gmail.com`  
- **GROQ_API_KEY** – from console.groq.com (free tier)

The code in Section 2 reads them via userdata.get(). Do not paste secrets in the notebook if you use Secrets.

**Option B – Paste in Section 2 cell**  
In the next code cell (Section 2), find the line that says `# PASTE YOUR VALUES BELOW` and replace the empty strings with your values. Do not share the notebook after pasting.

## 1. Install dependencies

In [7]:
!pip install -q yfinance pandas numpy feedparser requests python-dotenv schedule tenacity pytz psutil
!pip install -q python-telegram-bot
print('Dependencies installed.')

Dependencies installed.


## 1.5 Clone MNEMOS repository

When you open this notebook from GitHub ("Open in Colab"), only the notebook file is loaded—the repo is **not** cloned. This cell clones the project into `/content/Mnemos-2.0` so Section 2 finds the `engine` and other modules. No GitHub login needed (public repo).

In [ ]:
import os
import shutil

PROJECT_DIR = '/content/Mnemos-2.0'
if os.path.exists(os.path.join(PROJECT_DIR, 'main.py')):
    print('MNEMOS project already present at', PROJECT_DIR)
else:
    if os.path.exists(PROJECT_DIR):
        shutil.rmtree(PROJECT_DIR)
    !git clone https://github.com/harshyadavv2456/Mnemos-2.0.git /content/Mnemos-2.0
    if os.path.exists(os.path.join(PROJECT_DIR, 'main.py')):
        print('Cloned successfully.')
    else:
        raise SystemExit('Clone failed; check network and repo URL.')

## 2. Set project path and secrets

In [8]:
import os
from pathlib import Path

# When opened from GitHub (Open in Colab): repo is usually current dir or /content/Mnemos-2.0
MNEMOS_ROOT = Path('.')
if not (MNEMOS_ROOT / 'main.py').exists():
    MNEMOS_ROOT = Path('/content/Mnemos-2.0')
if not MNEMOS_ROOT.exists() or not (MNEMOS_ROOT / 'main.py').exists():
    MNEMOS_ROOT = Path('/content/mnemos2.0')
if not MNEMOS_ROOT.exists():
    MNEMOS_ROOT = Path('.')
os.chdir(MNEMOS_ROOT)
# Load .env from project if present (e.g. you zipped the project with .env)
from dotenv import load_dotenv
load_dotenv(MNEMOS_ROOT / '.env')
print('Working dir:', os.getcwd())

# ----- Colab Secrets: read via userdata.get(); enable Notebook access for each secret in sidebar -----
try:
    from google.colab import userdata
    for key in ('TELEGRAM_BOT_TOKEN', 'TELEGRAM_CHAT_ID', 'GMAIL_USER', 'GMAIL_APP_PASSWORD', 'ALERT_EMAIL_TO', 'GROQ_API_KEY'):
        try:
            os.environ[key] = userdata.get(key)
        except Exception:
            pass
except ImportError:
    pass
os.environ.setdefault('TELEGRAM_BOT_TOKEN', os.environ.get('TELEGRAM_BOT_TOKEN', ''))
os.environ.setdefault('TELEGRAM_CHAT_ID', os.environ.get('TELEGRAM_CHAT_ID', ''))
os.environ.setdefault('GMAIL_USER', os.environ.get('GMAIL_USER', ''))
os.environ.setdefault('GMAIL_APP_PASSWORD', os.environ.get('GMAIL_APP_PASSWORD', ''))
# One mail trail: comma-separated addresses (e.g. a@gmail.com,b@gmail.com)
os.environ.setdefault('ALERT_EMAIL_TO', os.environ.get('ALERT_EMAIL_TO', os.environ.get('GMAIL_USER', '')))
os.environ.setdefault('GROQ_API_KEY', os.environ.get('GROQ_API_KEY', ''))

ok = bool(os.environ.get('TELEGRAM_BOT_TOKEN') and os.environ.get('TELEGRAM_CHAT_ID') and os.environ.get('GMAIL_USER') and os.environ.get('GMAIL_APP_PASSWORD')); print('Secrets:', 'OK' if ok else 'MISSING – in Colab Secrets sidebar enable Notebook access for each secret, or paste values above.')

Working dir: /content
Secrets: MISSING – add Colab Secrets or paste above, then re-run.


## 3. Mount Google Drive (for DB backups + persistence)

In [9]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.insert(0, str(MNEMOS_ROOT))
from engine.orchestrator import set_drive_mount
set_drive_mount(Path('/content/drive/MyDrive'))
print('Drive mounted. Backups will go to Drive.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ModuleNotFoundError: No module named 'engine'

## 4. One-click: Run MNEMOS 24/7

In [ ]:
import sys
sys.path.insert(0, str(MNEMOS_ROOT))

from engine.orchestrator import run_forever
from storage.db import init_db

init_db()
print('MNEMOS 2.1 started. Heartbeats below; alerts to Telegram/Email. Daily heartbeat + weekly/monthly reports run automatically.')
print('To stop: Runtime -> Interrupt execution.')
# Use run_forever without supervisor so Colab does not restart the kernel on exception
run_forever(backup_interval_ticks=20, daily_task_interval_ticks=60)